# Performance Result Summary
Generate training time statistics from log files.

In [1]:
import os
from os import listdir
from os.path import isfile, join

import numpy as np
import pandas as pd
import re

from IPython.display import display

### Functions

In [2]:
def format_number_seq(l):
    """
    This functions find the sequence of consecutive numbers.
    Example: 
      input: [1,2,3,5,6]
      output: 1-3, 5-6
    
    l: a sequence of numbers, in numeric data type
    """
    l_res = []
    
    for i,v in enumerate(l):
        if i == 0:
            int_s = v
            int_e = None
        else:
            if v-l[i-1] == 1:
                pass
            else:
                int_e = l[i-1]
                l_res.append(f'{int_s}-{int_e}')
                int_s = v
                int_e = None
    if int_e is None:
        int_e = v
        l_res.append(f'{int_s}-{int_e}')
    return ', '.join(l_res)

In [3]:
def calculate_statistics(fn_or_dir,dir_log,type_log='f'):
    """
    fn_or_dir: file name for a log file, or directory name for a directory of logs from torchrun
    dir_log: a directory of all log files/dirs to search
    type_log: f / file or d / dir / directory
              f / file: one-file log
              d / dir / directory: directory of logs created by torchrun
    """
    if type_log in ['f','file']:
        with open(dir_log+'/'+fn_or_dir,'r') as f:
            text = f.readlines()
    
        tokens = fn_or_dir.replace('.log','').split()
    
    elif type_log in ['d', 'dir', 'directory']:
        text = []
        count = 0
        for attempt in os.listdir(dir_log+'/'+fn_or_dir):
            for worker in os.listdir(dir_log+'/'+fn_or_dir+'/'+attempt):
                with open(dir_log+'/'+fn_or_dir+'/'+attempt+'/'+worker+'/stdout.log','r') as f:
                    text = text + f.readlines()
                    count += 1
        print(f'[{type_log}] loaded {count} log files')
        
        tokens = fn_or_dir.split(';')[0].split()
    else:
        raise Exception(f'type_log should be either f / file, or d / dir / directory, not {type_log}')
        
    try:
        fn_type = tokens[1]
    except:
        fn_type = 'train'#tokens[0]
    
    messed_lines = [line for line in text if line.startswith('\x00')]
    if (len(messed_lines)>0):
        print(f'[{type_log}] {fn_or_dir} has {len(messed_lines)} lines messed up')
        
    if fn_type == 'train':
        print(f'[{type_log}] {fn_or_dir}')
        key_lines = [line for line in text if line.startswith('End')]
        

        pattern = '(\d+) sec'
        l_res_time = []
        for line in key_lines:
            l_res_time.append(int(re.findall(pattern, line)[0]))
            
        pattern = 'End of epoch (\d+)'
        l_res_epoch = []
        for line in key_lines:
            l_res_epoch.append(int(re.findall(pattern, line)[0]))
        epoch_ids_included = format_number_seq(l_res_epoch)

        dict_res = {'count':len(l_res_time),
                    'epochs': epoch_ids_included,
                    'mean': np.mean(l_res_time),
                    'std': np.std(l_res_time),
                    'min': np.min(l_res_time),
                    'max': np.max(l_res_time)}
        dict_res['projected_time_s'] = dict_res['mean'] * 200
        dict_res['projected_time_h'] = dict_res['projected_time_s'] / 60 / 60
        dict_res['params'] = tokens[3] if len(tokens) > 3 and tokens[3] not in ['slow','fast'] else ''
        dict_res['node'] = tokens[-1] if tokens[-1] in ['slow','fast'] else 'unknown'
#         dict_res['text'] = key_lines
        return dict_res

### Main

In [4]:
dir_log = '/userfs/log'

fns = [fn for fn in listdir(dir_log) if isfile(join(dir_log, fn))]
fns = [fn for fn in fns if fn.endswith('.log') and fn.startswith('')]

dirs = [d for d in listdir(dir_log) if not isfile(join(dir_log,d))]
dirs = [d for d in dirs if len(d.split(';'))==2]

display(fns)
display(dirs)

['wmla train 20210102 num_threads0,batch_size2,num_workers1 slow.log',
 'ws gpu 20211121 num_threads0 fast.log',
 'train-oldgit.log',
 'ws train 20211123 gpu2 slow.log',
 'wmla train 20210104 num_threads0,batch_size2,num_workers3.log',
 'wmla train 20210101 num_threads0,batch_size2,num_workers2 slow.log',
 'geckodriver.log',
 'gpu.log',
 'ws train 20211124 gpu2,batch_size4 slow.log',
 'wmla train 20210102 num_threads0,batch_size2,num_workers3.log',
 'ws train 20211123 batch_size2 slow.log',
 'ws train 20211121 num_threads0 fast.log',
 'wmla train 20210103 num_threads0,batch_size2,num_workers3.log',
 'ws train 20211201 num_threads0 fast.log',
 'ws train 20211124 ??? fast.log',
 'train.log',
 'ws gpu 20211123 batch_size2.log']

['ws train 20220102 num_threads0,batch_size2,num_workers1 slow;_5a_xge_8',
 'ws train 20220101 num_threads0,batch_size2,num_workers2 fast;__6vd5hvg']

In [5]:
dict_stat = {}
for fn in fns:
    if ' train ' in fn:
        dict_stat[fn] = calculate_statistics(fn,dir_log)
        
for d in dirs:
    dict_stat[d] = calculate_statistics(d,dir_log,type_log='d')

df_res = pd.DataFrame(dict_stat).T
df_res

[f] wmla train 20210102 num_threads0,batch_size2,num_workers1 slow.log
[f] ws train 20211123 gpu2 slow.log
[f] wmla train 20210104 num_threads0,batch_size2,num_workers3.log
[f] wmla train 20210101 num_threads0,batch_size2,num_workers2 slow.log
[f] ws train 20211124 gpu2,batch_size4 slow.log
[f] wmla train 20210102 num_threads0,batch_size2,num_workers3.log
[f] ws train 20211123 batch_size2 slow.log
[f] ws train 20211121 num_threads0 fast.log has 1 lines messed up
[f] ws train 20211121 num_threads0 fast.log
[f] wmla train 20210103 num_threads0,batch_size2,num_workers3.log
[f] ws train 20211201 num_threads0 fast.log
[f] ws train 20211124 ??? fast.log
[d] loaded 1 log files
[d] ws train 20220102 num_threads0,batch_size2,num_workers1 slow;_5a_xge_8
[d] loaded 2 log files
[d] ws train 20220101 num_threads0,batch_size2,num_workers2 fast;__6vd5hvg


,count,epochs,mean,std,min,max,projected_time_s,projected_time_h,params,node
"wmla train 20210102 num_threads0,batch_size2,num_workers1 slow.log",74,0-73,517.649,4.23445,512,545,103530,28.7583,"num_threads0,batch_size2,num_workers1",slow
ws train 20211123 gpu2 slow.log,69,0-68,519.87,3.2656,515,533,103974,28.8816,gpu2,slow
"wmla train 20210104 num_threads0,batch_size2,num_workers3.log",21,"0-6, 0-6, 0-6",292.381,143.621,207,646,58476.2,16.2434,"num_threads0,batch_size2,num_workers3",unknown
"wmla train 20210101 num_threads0,batch_size2,num_workers2 slow.log",402,"0-200, 0-200",259.107,3.85767,253,298,51821.4,14.3948,"num_threads0,batch_size2,num_workers2",slow
"ws train 20211124 gpu2,batch_size4 slow.log",201,0-200,416.1,4.93742,409,453,83219.9,23.1166,"gpu2,batch_size4",slow
"wmla train 20210102 num_threads0,batch_size2,num_workers3.log",603,"0-200, 0-200, 0-200",176.755,20.863,169,471,35350.9,9.8197,"num_threads0,batch_size2,num_workers3",unknown
ws train 20211123 batch_size2 slow.log,113,0-112,526.292,3.56403,521,539,105258,29.2384,batch_size2,slow
ws train 20211121 num_threads0 fast.log,44,157-200,602.432,4.1253,597,613,120486,33.4684,num_threads0,fast
"wmla train 20210103 num_threads0,batch_size2,num_workers3.log",97,"0-31, 0-31, 0-32",183.175,50.757,169,470,36635.1,10.1764,"num_threads0,batch_size2,num_workers3",unknown
ws train 20211201 num_threads0 fast.log,126,0-125,600.833,4.23281,595,619,120167,33.3796,num_threads0,fast


In [6]:
df_res[df_res['params'].str.contains('num_workers')]

,count,epochs,mean,std,min,max,projected_time_s,projected_time_h,params,node
"wmla train 20210102 num_threads0,batch_size2,num_workers1 slow.log",74,0-73,517.649,4.23445,512,545,103530,28.7583,"num_threads0,batch_size2,num_workers1",slow
"wmla train 20210104 num_threads0,batch_size2,num_workers3.log",21,"0-6, 0-6, 0-6",292.381,143.621,207,646,58476.2,16.2434,"num_threads0,batch_size2,num_workers3",unknown
"wmla train 20210101 num_threads0,batch_size2,num_workers2 slow.log",402,"0-200, 0-200",259.107,3.85767,253,298,51821.4,14.3948,"num_threads0,batch_size2,num_workers2",slow
"wmla train 20210102 num_threads0,batch_size2,num_workers3.log",603,"0-200, 0-200, 0-200",176.755,20.863,169,471,35350.9,9.8197,"num_threads0,batch_size2,num_workers3",unknown
"wmla train 20210103 num_threads0,batch_size2,num_workers3.log",97,"0-31, 0-31, 0-32",183.175,50.757,169,470,36635.1,10.1764,"num_threads0,batch_size2,num_workers3",unknown
"ws train 20220102 num_threads0,batch_size2,num_workers1 slow;_5a_xge_8",48,0-47,506.604,4.54945,502,532,101321,28.1447,"num_threads0,batch_size2,num_workers1",slow
"ws train 20220101 num_threads0,batch_size2,num_workers2 fast;__6vd5hvg",402,"0-200, 0-200",259.933,3.68984,251,284,51986.6,14.4407,"num_threads0,batch_size2,num_workers2",fast


In [7]:
df_res[~df_res['params'].str.contains('num_workers')]

,count,epochs,mean,std,min,max,projected_time_s,projected_time_h,params,node
ws train 20211123 gpu2 slow.log,69,0-68,519.87,3.2656,515,533,103974,28.8816,gpu2,slow
"ws train 20211124 gpu2,batch_size4 slow.log",201,0-200,416.1,4.93742,409,453,83219.9,23.1166,"gpu2,batch_size4",slow
ws train 20211123 batch_size2 slow.log,113,0-112,526.292,3.56403,521,539,105258,29.2384,batch_size2,slow
ws train 20211121 num_threads0 fast.log,44,157-200,602.432,4.1253,597,613,120486,33.4684,num_threads0,fast
ws train 20211201 num_threads0 fast.log,126,0-125,600.833,4.23281,595,619,120167,33.3796,num_threads0,fast
ws train 20211124 ??? fast.log,201,0-200,576.299,4.75368,570,616,115260,32.0166,???,fast
